In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import re
import spacy
!python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")
print("loaded")

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-10 20:31:03.253089: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
loaded


In [3]:
import pandas as pd

In [4]:
df = pd.read_excel("/content/drive/MyDrive/Ashkan/output_anonomization_no_names_only_english_version2.xlsx")

In [5]:
print(df.shape)
filt = df['isEnglish'] == 1
df = df.loc[filt]
df.shape

(67171, 37)


(67171, 37)

In [6]:
df.columns

Index(['Unnamed: 0', 'number', 'opened_by', 'short_description', 'priority',
       'assignment_group', 'assigned_to', 'u_main_category_reporting',
       'u_subcategory_1_reporting', 'u_subcategory_2_reporting',
       'u_subcategory_3_reporting', 'u_resolver_group', 'u_resolver',
       'u_first_assignment_group', 'u_country', 'sys_updated_on',
       'sys_updated_by', 'sys_mod_count', 'u_vip', 'u_type', 'u_region',
       'contact_type', 'location', 'u_resolved', 'closed_at',
       'u_assignment_group_history', 'work_notes', 'comments', 'close_notes',
       'u_assignee_history', 'merged', 'merged_string', 'business_duration',
       'u_reassignment_count_assigne', 'reassignment_count', 'cleaned_string',
       'isEnglish'],
      dtype='object')

In [7]:
def remove_lines_with_date_start(text):
    try:
        splitted_text = text.split("\n")
    except:
        splitted_text = []
    new_text = ""
    for i, sp_text in enumerate(splitted_text):
        if not re.match("[0-9]{4}-[0-9]{2}-[0-9]{2} [0-9]{2}[0-9]{2}[0-9]{2}", sp_text[0:19]):
            new_text += sp_text + "\n"
        else:
            if not re.match("[0-9]{4}-[0-9]{2}-[0-9]{2} [0-9]{2}[0-9]{2}[0-9]{2}", splitted_text[i - 1]):
                new_text += "______________________________" + "\n"
            else:
                new_text += "\n"
                
    return new_text.rstrip()


def remove_lines_start(text, start):
    greetings = ["good day", "hi", "hello", "dear", "many thanks", "thank you"]
    try:
        splitted_text = text.split("\n")
    except:
        splitted_text = []
    new_text = ""
    for sp_text in splitted_text:
        cond = True
        if not sp_text[0:len(start)] == start:
            clean_sp_text = ''.join([d for d in sp_text if not d.isdigit()])
            clean_sp_text = clean_sp_text.replace('.', '')
            clean_sp_text = clean_sp_text.replace('-', '')
            clean_sp_text_list = list(filter(None, clean_sp_text.strip().split(" ")))
            # print(clean_sp_text_list)
            if len(clean_sp_text_list) <= 5:
                for greeting in greetings:
                    if clean_sp_text.lower().strip().startswith(greeting):
                        cond = False
                if len(clean_sp_text_list) <= 1:
                    cond = False

            if sp_text == "______________________________":
                cond = True
            if cond:
                new_text += sp_text + "\n"

    return new_text.rstrip()


def preprocess(ticket):
    ticket = '\n' + str(ticket)
    ticket_list = ticket.split(" ")

    # remove names after @
    for i, w in enumerate(ticket):
        if w == '@':
            y = ticket.find(' ', i)
            z = ticket.find(' ', y+1)
            ticket = ticket.replace(ticket[i:z], '')

    index = [m.start() for m in re.finditer(re.escape('code]'), ticket)]
    if len(index) > 0:
        for i, ind in enumerate(index):
            try:
                index = [m.start() for m in re.finditer(re.escape('code]'), ticket)]
                start_index = index[0] - 1
                end_index = index[1] + 5
                ticket = ticket[:start_index] + ticket[end_index + 1:]
                index = [m.start() for m in re.finditer(re.escape('code]'), ticket)]
                if len(index) < 2:
                    break
            except:
                continue

    extras = ['[', ']', 'cid', '+', '(', ')', ':', "xD",
              '<', '>', '#', '~', '/', '{', '}', "Issue description", "_x000D_", ';', ',', "Pvt  Limited", "Short Description"
              , "My action taken", "Issues", "Action taken", "Directory Services Administrator", "PM", "AM", "nan"]

    for extra in extras:
        ticket = ticket.replace(extra, '')

    text = "The information contained in this communication"
    if text in ticket:
        extra_text_index = ticket.find("The information contained in this communication")
        extra_lines = ticket[extra_text_index:].split('\n')[:9]
        for line in extra_lines:
            ticket = ticket.replace(line, '')

    ticket = re.sub('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+', '', ticket)

    ticket = ticket = '\n'.join(line.strip() for line in ticket.split('\n') if line != '' and "khayaban" not in line.lower())
    line_beginnings_remove = ["Additional Location Information", "Affected Device", "received from", "Email",
                              "Tel", "Subject", "From", "Sent", "To", "reply from", "Issue(s)"
                              , "Who called", "Ext Number", "Locations", "Device", "Call time", "Computer", "Client Type", "Version VPN", "PC Name", "Keyword",
                              "Sent", "From"]


    for beginning in line_beginnings_remove:
        ticket = remove_lines_start(ticket, beginning)

    ticket = remove_lines_with_date_start(ticket)

    doc = nlp(ticket)
    names = [i for i in doc.ents if i.label_.lower() in ["person", "gpe", "location"]]
    for name in names:
        ticket = ticket.replace(str(name), '')

    ticket = '\n '.join(line for i, line in enumerate(ticket.split('\n')) if line != '' and (1 < i < len(ticket.split('\n')) - 1 or line != "______________________________"))

    return ticket.strip()

In [8]:
clean_comments = []
clean_work = []
clean_close = []

for index, row in df.iterrows():
    comment = str(row['short_description']) + '\n' + str(row['comments'])
    clean_comments.append(preprocess(comment))
    
    w = str(row['work_notes'])
    clean_work.append(preprocess(w))
   

    c = str(row['close_notes'])
    clean_close.append(preprocess(c))
   
    if index%1000==0:
        print(index)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000


In [9]:
df['clean_work_notes'] = clean_work
df['clean_comments'] = clean_comments
df['clean_close_notes'] = clean_close

In [10]:
df.to_excel('/content/drive/MyDrive/Ashkan/output_clean_comments_no_name_version2.xlsx')